# LUPA

In [ ]:
import gmsh, pygmsh
import capytaine as cpy
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import wecopttool as wot

In [ ]:
rho = 1000

## BEM Coefficients

### Buoy

In [ ]:
# mesh
mesh_size_factor = 0.3
r1 = 1.0/2
r2 = 0.4/2
h1 = 0.5
h2 = 0.21
freeboard = 0.3
r3 = 0.102/2 + 0.05  # TODO: geometry

with pygmsh.occ.Geometry() as geom:
    gmsh.option.setNumber('Mesh.MeshSizeFactor', mesh_size_factor)
    cyl = geom.add_cylinder([0, 0, 0], [0, 0, -h1], r1)
    cone = geom.add_cone([0, 0, -h1], [0, 0, -h2], r1, r2)
    geom.translate(cyl, [0, 0, freeboard])
    geom.translate(cone, [0, 0, freeboard])
    tmp = geom.boolean_union([cyl, cone])
    bar = geom.add_cylinder([0, 0, 10], [0,0,-20], r3)
    geom.boolean_difference(tmp, bar)
    mesh_buoy = geom.generate_mesh()

In [ ]:
# floating body
buoy_fb = cpy.FloatingBody.from_meshio(mesh_buoy, name='buoy')
buoy_fb.add_translation_dof(name='Heave')

In [ ]:
# mass properties
mass_buoy = buoy_fb.disp_mass(rho=rho)
cm_buoy = buoy_fb.center_of_buoyancy  # TODO: mass properties

buoy_fb_copy = buoy_fb.copy()
buoy_fb_copy.keep_only_dofs([])
buoy_fb_copy.center_of_mass = cm_buoy
buoy_fb_copy.rotation_center = buoy_fb_copy.center_of_mass
buoy_fb_copy.add_rotation_dof(name="Pitch")
pitch_inertia_buoy = buoy_fb_copy.compute_rigid_body_inertia(rho=rho).values[0, 0]  # TODO: mass properties

In [ ]:
# show
buoy_fb.show_matplotlib()
# buoy_fb.show()

### Spar

In [ ]:
# mesh
mesh_size_factor = 0.3
r1 = 0.445/2  # body
r2 = 0.914/2  # plate
h1 = 1.016 
h2 = 0.05  # TODO: geometry
h3a = 1.651
submergence = 3.658 - h3a - h1 - h2
r3 = 0.102/2  # bar

with pygmsh.occ.Geometry() as geom:
    gmsh.option.setNumber('Mesh.MeshSizeFactor', mesh_size_factor)
    body = geom.add_cylinder([0, 0, 0], [0, 0, -h1], r1)
    geom.translate(body, [0, 0, -submergence])
    plate = geom.add_cylinder([0, 0, 0], [0, 0, -h2], r2)
    geom.translate(plate, [0, 0, -(submergence+h1)])
    bar = geom.add_cylinder([0, 0, h3a], [0, 0, -(h3a+submergence)], r3)
    geom.boolean_union([bar, body, plate])
    mesh_spar = geom.generate_mesh()

In [ ]:
# floating body
spar_fb = cpy.FloatingBody.from_meshio(mesh_spar, name='spar')
spar_fb.add_translation_dof(name='Heave')

In [ ]:
# mass properties
mass_spar = spar_fb.disp_mass(rho=rho)
cm_spar = spar_fb.center_of_buoyancy  # TODO: mass properties

spar_fb_copy = spar_fb.copy()
spar_fb_copy.keep_only_dofs([])
spar_fb_copy.center_of_mass = cm_spar
spar_fb_copy.rotation_center = spar_fb_copy.center_of_mass
spar_fb_copy.add_rotation_dof(name="Pitch")
pitch_inertia_spar = spar_fb_copy.compute_rigid_body_inertia(rho=rho).values[0, 0]  # TODO: mass properties

In [ ]:
# show
spar_fb.show_matplotlib()
# spar_fb.show()

### LUPA

In [ ]:
# floating body # mass properties
lupa_fb = buoy_fb + spar_fb
lupa_fb.name = 'LUPA'
lupa_fb.add_translation_dof(name='Surge')
lupa_fb.add_rotation_dof(name='Pitch')
lupa_fb.center_of_mass = (mass_buoy*cm_buoy + mass_spar*cm_spar) / (mass_buoy + mass_spar)
lupa_fb.rotation_center = lupa_fb.center_of_mass
_ = lupa_fb.compute_hydrostatics(rho=rho)

In [ ]:
# show
lupa_fb.show_matplotlib()
# lupa_fb.show()

### BEM

In [ ]:
# compute coefficients
f1 = 0.01
nfreq = 50
freq = wot.frequency(f1, nfreq, False)
bem_data = wot.run_bem(lupa_fb, freq)

In [ ]:
# plot coefficients
radiating_dof = "buoy__Heave"
# radiating_dof = "spar__Heave"
# radiating_dof = "Surge"
# radiating_dof = "Pitch"

influenced_dof = "buoy__Heave"
# influenced_dof = "spar__Heave"
# influenced_dof = "Surge"
# influenced_dof = "Pitch"

# added mass
plt.figure()
bem_data.added_mass.sel(radiating_dof=radiating_dof, influenced_dof=influenced_dof).plot()

# radiation damping
plt.figure()
bem_data.radiation_damping.sel(radiating_dof=radiating_dof, influenced_dof=influenced_dof).plot()

# diffraction
plt.figure()
np.abs(bem_data.diffraction_force.sel(influenced_dof=influenced_dof)).plot()

# FK
plt.figure()
np.abs(bem_data.Froude_Krylov_force.sel(influenced_dof=influenced_dof)).plot()

## WEC Object

In [ ]:
# inertia & hydrostatics
hydrostatic_stiffness = lupa_fb.hydrostatic_stiffness

d_buoy = cm_buoy[2] - lupa_fb.center_of_mass[2]
d_spar = cm_spar[2] - lupa_fb.center_of_mass[2]
pitch_inertia = (
    pitch_inertia_buoy + mass_buoy*d_buoy**2 + 
    pitch_inertia_spar + mass_spar*d_spar**2
)
inertia = np.diag([mass_buoy, mass_spar, lupa_fb.disp_mass(), pitch_inertia])

In [ ]:
# PTO
def gear_ratio(pulley_radius):
    return 1/pulley_radius  # rad/m 

def pto_impedance(pulley_radius, omega=bem_data.omega.values):
    torque_constant = 8.51  # N*m/A
    winding_resistance = 5.87  # Ω 
    winding_inductance = 0.0536  # H 
    drivetrain_inertia = 2.0 / 2  # Kg*m^2 # TODO: rotor inertia 1.8e-2 + inertia of other 3 pulleys 
    drivetrain_friction = 1.0 / 2  # N*m*s/rad # TODO
    drivetrain_stiffness = 0.0  # N*m/rad 
    drivetrain_impedance = (1j*omega*drivetrain_inertia + 
                            drivetrain_friction + 
                            1/(1j*omega)*drivetrain_stiffness) 
    winding_impedance = winding_resistance + 1j*omega*winding_inductance
    pto_impedance_11 = -1* gear_ratio(pulley_radius)**2 * drivetrain_impedance
    off_diag = -1*np.ones(omega.shape) * (
        np.sqrt(3.0/2.0) * torque_constant * gear_ratio(pulley_radius) + 0j)
    pto_impedance_12 = off_diag 
    pto_impedance_21 = off_diag
    pto_impedance_22 = winding_impedance
    impedance = np.array([[pto_impedance_11, pto_impedance_12],
                          [pto_impedance_21, pto_impedance_22]])
    return impedance

name = ["PTO_Heave",]
kinematics = np.array([[1, -1, 0, 0],])
pto_ndof = 1
controller = None
loss = None
min_radius = 0.0815
max_radius = 0.2037
mid_radius = 0.1273
default_radii = [min_radius, mid_radius, max_radius] 
pto = wot.pto.PTO(pto_ndof, kinematics, controller, pto_impedance(mid_radius), loss, name)

In [ ]:
# Constraints
constraints = None  # TODO
nsubsteps = 5

## Geometry/displacements
# minimum relative position between buoy and spar (soft end stop)
stroke_max = 0.5  # m
def const_stroke_pto(wec, x_wec, x_opt, waves): 
    pos = pto.position(wec, x_wec, x_opt, waves, nsubsteps)
    return pos - np.abs(stroke_max.flatten())

# maximum position of spar: (hit top beam?)
# minimum position of spar: (hit bottom?)

## GENERATOR
# peak torque (generator) 
torque_peak = 137.9  # N*m
def const_peak_torque_pto(wec, x_wec, x_opt, waves): 
    """Instantaneous torque must not exceed max torque Tmax - |T| >=0 
    """
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps) / gear_ratio(mid_radius)
    return torque_peak - np.abs(torque.flatten())

# continuous torque (generator)
torque_continuous = 46  # N*M
def const_torque_pto(wec, x_wec, x_opt, waves): 
    """RMS torque must not exceed max continous torque 
        Tmax_conti - Trms >=0 """
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps) / gear_ratio(mid_radius)
    torque_rms = np.sqrt(np.mean(torque**2))
    return torque_continuous - np.abs(torque_rms.flatten())

# max speed (generator)
rot_speed = 150  #rpm
rot_speed *= 2*np.pi  # rad/s
def const_speed_pto(wec, x_wec, x_opt, waves): 
    rot_vel = pto.velocity(wec, x_wec, x_opt, waves, nsubsteps) * gear_ratio(mid_radius)
    return rot_speed - np.abs(rot_vel.flatten())

In [ ]:
# Additional Forces

# mooring
M = np.zeros([4,4])  # TODO
moor = ((M + 0j).expand_dims({"omega": bem_data.omega}))
tmp = moor.isel(omega=0).copy(deep=True)
tmp['omega'] = tmp['omega'] * 0
moor = xr.concat([tmp, moor], dim='omega') 
moor = moor.transpose("radiating_dof", "influenced_dof", "omega")
moor = -1*moor  # RHS of equation: ma = Σf 
mooring_force = wot.force_from_rao_transfer_function(moor, True)

f_add = {
    'PTO': pto.force_on_wec,
    'Mooring': mooring_force
}

friction = np.diag([0, 0, 0, 0])

In [ ]:
# WEC
wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=inertia,
                       hydrostatic_stiffness=hydrostatic_stiffness,
                       constraints=constraints,
                       friction=friction,
                       f_add=f_add
)

## Waves

In [ ]:
# regular (test/setup)
amplitude = 0.1  
wavefreq = 0.5
phase = 0
wavedir = 0
waves_reg = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

# LWF (South)
cases = {
    'max_90': {'Hs': 0.21, 'Tp': 3.09}, 
    'max_annual': {'Hs': 0.13, 'Tp': 2.35},
    'max_occurrence': {'Hs': 0.07, 'Tp': 1.90},
    'min_10': {'Hs': 0.04, 'Tp': 1.48},  
}

case = cases['max_occurence']  # CHANGE HERE! 
hs = cases['Hs']
fp = 1/cases['Tp']
spectrum = lambda f: wot.waves.jonswap_spectrum(f, fp, hs, gamma=3.3)
efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
waves = wot.waves.long_crested_wave(efth)

## Solve

In [ ]:
# Objective function
obj_fun = pto.average_power
nstate_opt = wec.ncomponents

# Solve
scale_x_wec = 1e1  
scale_x_opt = 1e-3  
scale_obj = 1e-2  

results = wec.solve(
    waves_reg,  # CHANGE HERE 
    obj_fun, 
    nstate_opt, 
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
)

print(f'Optimal average electrical power: {-results.fun} W')

# Post-process
nsubsteps = 5
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubsteps)

## Results